In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import os

//anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [44]:

#Model - this makes the network (ie the policy)

#we have the soft Q function, the parameterised value function V, and the parameterised tractable policy pi

# TODOs
# Make comments properly

# Whitening of inputs

# Train, test routines


class replayBuffer():
    def __init__(self,n_inputs,n_outputs,max_buffer_size = 1e4,min_pool_size=1000,batch_size=128):
        self._max_size = int(max_buffer_size)
        self._min_pool_size = int(min_pool_size)
        self._batch_size = batch_size
        self.n_outputs = n_outputs
        self.n_inputs = n_inputs
        self.reset()
        
    def reset(self):
        self._size = 0
        self._pos = 0
        
        self.actions = np.zeros([self._max_size,self.n_outputs])
        self.observations = np.zeros([self._max_size,self.n_inputs])
        self.next_observations = np.zeros([self._max_size,self.n_inputs])
        self.rewards = np.zeros(self._max_size)
        self.dones = np.zeros(self._max_size)
        
    def add_sample(self,action,obs,next_obs,reward,done):
        self.actions[self._pos] = action
        self.observations[self._pos] = obs
        self.next_observations[self._pos] = next_obs
        self.rewards[self._pos] = reward
        self.dones[self._pos] = done
        
        self._advance()
    
    def _advance(self):
        self._pos = (self._pos + 1) % self._max_size
        
        if self._size < self._max_size:
            self._size += 1
            
    def get_samples(self):
        inds = np.random.randint(0,self._size,self._batch_size)
        return dict(actions = self.actions[inds],
                   observations = self.observations[inds],
                   next_observations = self.next_observations[inds],
                   rewards = self.rewards[inds],
                   dones = self.dones[inds])
    
    def get_last_sample(self):
        last_pos = [(self._pos-1) % self._max_size]
        return dict(actions = self.actions[last_pos],
                   observations = self.observations[last_pos],
                   next_observations = self.next_observations[last_pos],
                   rewards = self.rewards[last_pos],
                   dones = self.dones[last_pos])
    
    def batch_ready(self):
        return self._size >= self._min_pool_size
        
class logger():
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.vars = {}
        
    def record(self,var_name,value):
        if hasattr(self.vars,var_name):
            self.vars[var_name].append(value)
        else:
            self.vars[var_name] = [value]
    
    def get(self,var_name):
        if hasattr(self.vars,var_name):
            return self.vars[var_name]
        else:
            return False

class Sampler():
    def __init__(self,policy,env,replaybuffer):
        self.policy = policy
        self.env = env
        self.replaybuffer = replaybuffer
        
        self.reset()
        
    def reset(self): 
        self.current_obs = False
        self.episodes = 0
        self.current_episode_reward = 0
        self.mean_episode_reward = 0

    def sample(self):
        if (self.current_obs is False):
            self.current_obs = env.reset()
            
        action = self.policy.get_action(self.current_obs)
        next_obs, reward, done, info = env.step(action)
        if self.policy.discrete == True:
            action = np.eye(self.policy.action_size)[action]
        rb.add_sample(action,self.current_obs,next_obs,reward,done)
        self.current_obs = next_obs
        
        self.current_episode_reward += reward

        if done:
            self.current_obs = False   
            self.episodes += 1

            log.record('episode_reward',self.current_episode_reward)
            self.mean_episode_reward = (self.mean_episode_reward * (self.episodes - 1) + self.current_episode_reward) / self.episodes
            self.current_episode_reward = 0
            


In [45]:

# Here is a helper class to make a simple neural network. Importantly, it allows us to easily get the parameters, and hopefully to link the inputs to other variables
# The get_output functionality is borrowed from the SAC reference code.

class MLP():
    def __init__(self,name,inputs,output_size,n_hidden,n_layers):
        self._name = name
        self.inputs = inputs
        self.output_size = output_size
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        
        self.output = self.make_network(reuse = False)
        
    def make_network(self,inputs = False,reuse = tf.AUTO_REUSE):
        # This function just makes a simple fully connected network. It is structured in a little bit of a silly way. The idea is that this lets one reuse the network weights elsewhere with different inputs. Currently not actually using this functionality 
        if inputs is False :
            inputs = self.inputs
            
        with tf.variable_scope(self._name,reuse = reuse):
            if not(isinstance(inputs,tf.Tensor)):  # Can chuck in more than one input. This just concatenates them
                inputs = tf.concat(inputs,axis=1)

            # To do: understand weight initialization!   
            self.hidden = slim.stack(inputs, slim.fully_connected, [self.n_hidden]*self.n_layers, scope='fc',activation_fn=tf.nn.relu) #,weights_regularizer=slim.l2_regularizer(0.1)
            outputs = slim.fully_connected(self.hidden,self.output_size,activation_fn=None)
        return outputs

    def get_params_internal(self):
        # Useful function to get network weights
        
        scope = tf.get_variable_scope().name
        scope += '/' + self._name + '/' if len(scope) else self._name + '/'

        return tf.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope
        )

In [46]:
class Qnet(MLP):
    # Make a simple q network
    def __init__(self,action_size,obs,n_hidden,n_layers):
        # Ok so Q function takes s,a, gives Q   
        self.obs = obs
        # Super is used to call the init method of the parent class
        super(Qnet,self).__init__('qNet',self.obs,action_size,n_hidden,n_layers)
      

In [108]:

class Policy_Discrete():
    # Make a policy.
    # For now assume that discrete action space, such that tractable - obviously this slightly defeats the point of SAC implementation since all about how hard to compute the partition function
    # P function takes s, needs to be able to give actions.

    def __init__(self,Qnet,scheme = 'Bltz',reward_scale = 1.0,epsilon_start = 1,epsilon_end=0.1,epsilon_decay=10000):

        self.Qnet = Qnet
        self.action_size = Qnet.output_size
        self.reward_scale = reward_scale
        self.scheme = scheme
        self.e = epsilon_start
        self.e_end = epsilon_end
        self.e_decay_frac = (1-1/epsilon_decay)
        
        self.discrete = True
        self._name = 'Policy'
        self.make_policy_outputs(reuse=False)
        
    def make_policy_outputs(self, reuse = tf.AUTO_REUSE):
       
        with tf.variable_scope(self._name,reuse = reuse):
            if self.scheme == 'Bltz':
                self.policy_output = tf.nn.softmax(reward_scale*self.Qnet.output,axis=1) # Automatically sum to one.
                self.log_policy_output = tf.log(self.policy_output)
                self.action = tf.multinomial(self.log_policy_output, num_samples=1)[0] # Will generate an action
            elif self.scheme == 'Epsilon':
                self.action = tf.argmax(self.Qnet.output)
                
            
    def get_action(self,obs):
        
        if self.scheme == 'Bltz':
                a = self.action.eval(feed_dict = {self.Qnet.obs : [obs]})[0]
                
        elif self.scheme == 'Epsilon':
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if np.random.rand(1) < self.e:
                a = np.random.randint(0,self.action_size)
            else:
                a = self.action.eval(feed_dict = {self.Qnet.obs : [obs]})[0]
                
            self.e = (self.e - self.e_end) * self.e_decay_frac + self.e_end
            
        return a
                

    
    
class Deep_Q_Learning():
    # This class handles the training of the networks
    def __init__(self,Qnet,actions,obs,next_obs,rewards,dones,lr=3e-4,discount = 0.99, tau=0.005):
        self.lr = lr
        self.discount = discount
        self.tau = tau 
        
        # Maybe would be nicer to not pass these but define here, but this seems to be messy. Once check if works, could go back to defining here
        self.actions = actions
        self.obs = obs
        self.next_obs = next_obs
        self.rewards = rewards
        self.dones = dones
        
        self.Qnet = Qnet
        self.Q_outputs = Qnet.output
        
        with tf.variable_scope('qNet_T'):
            self.target_Q_outputs = Qnet.make_network(inputs = self.next_obs,reuse=False) 
            self.target_Q_params = Qnet.get_params_internal()
        
        self.next_obs_out = Qnet.make_network(inputs = self.next_obs)
        self.predict = tf.one_hot(tf.argmax(Qnet.make_network(inputs = self.next_obs),axis=1),self.Qnet.output_size)
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate = self.lr)
        
        self.train_ops = []
        self.init_Q_net_training()
        self.init_target_Q_update()
        
        
    def init_Q_net_training(self):
        training_variables = self.Qnet.get_params_internal()
        with tf.variable_scope('Q_loss'):
            Q_t = tf.stop_gradient(self.rewards +  self.discount * (1-self.dones) * tf.reduce_sum(self.target_Q_outputs*self.predict,axis=1))
            self.Q_Loss = 0.5*tf.reduce_mean(tf.square(tf.reduce_sum(self.Q_outputs*self.actions,axis=1) - Q_t))
            tf.summary.scalar('Q_loss', self.Q_Loss)
        

#         Qnet_regularization_losses = tf.get_collection(
#             tf.GraphKeys.REGULARIZATION_LOSSES,
#             scope=self.Qnet._name)
#         Qnet_regularization_loss = tf.reduce_sum(
#             Qnet_regularization_losses)
    
#         gradients, variables = zip(*self.optimizer.compute_gradients(self.Q_Loss + Qnet_regularization_loss,var_list = training_variables))
#         gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
#         self.train_Q = self.optimizer.apply_gradients(zip(gradients, variables))

        self.train_Q = self.optimizer.minimize(self.Q_Loss,var_list = training_variables)

        self.train_ops.append(self.train_Q)
        
    def init_target_Q_update(self):
        # Pull the qnet params
        qnet_params = self.Qnet.get_params_internal()
        
        with tf.variable_scope('Target_Q_update'):
            self.tQnet_update = []
            for tQ_p in self.target_Q_params:
                # Match each target net param with equiv from vnet
                Q_p = [v for v in qnet_params if tQ_p.name[(tQ_p.name.index('/')+1):] in v.name]
                assert(len(Q_p) == 1) # Check that only found one variable
                Q_p = Q_p[0]
                with tf.control_dependencies([self.train_Q]):
                    self.tQnet_update.append(tQ_p.assign(self.tau * Q_p + (1-self.tau)*tQ_p))
            self.tQnet_update = tf.group(self.tQnet_update)
            
        self.train_ops.append(self.tQnet_update)
        
    def _construct_feed_dict(self,samples):  
        return {self.actions : samples['actions'],
                    self.obs : samples['observations'],
                    self.next_obs : samples['next_observations'],
                    self.dones : samples['dones'],
                    self.rewards : samples['rewards']}
                    
    def train(self, samples, *args):
        feed_dict = self._construct_feed_dict(samples)
        return tf.get_default_session().run([self.train_ops] + list(args), feed_dict = feed_dict)[1:]


In [109]:
params = {
    
    'CartPole-v0' : 
    {'base' : dict(
        env_name = 'CartPole-v0',
        epoch_length = 1000,
        max_epochs = 100,
        online_training = True,
        grad_steps_per_t = 1,
        ),
     'replay_buffer' : dict(
        batch_size = 100,
        max_buffer_size = 50000,
        min_pool_size = 500,
     ),
     'algorithm' : dict(
        reward_scale = 1,
        epsilon_start = 1.0,
        epsilon_end = 0.1,
        epsilon_decay = 20000,
        scheme = 'Epsilon',
        lr = 1e-5,
        tau = 0.01
     ),
     'nnet' : dict(
        n_hidden = 10,
        n_layers = 1
     )
    },
    
         'LunarLander-v2' : 
    {'base' : dict(
        env_name = 'LunarLander-v2',
        epoch_length = 1000,
        max_epochs = 100,
        online_training = False,
        grad_steps_per_t = 1,
        ),
     'replay_buffer' : dict(
        batch_size = 256,
        max_buffer_size = 1e6,
        min_pool_size = 1000,
     ),
     'algorithm' : dict(
        reward_scale = 2,
        epsilon_start = 1.0,
        epsilon_end = 0.1,
        epsilon_decay = 10000,
        scheme = 'Epsilon',
        lr = 3e-4,
        tau = 0.01
     ),
     'nnet' : dict(
        n_hidden = 20,
        n_layers = 2
     )
    }
    
    
}

In [110]:
log_dir = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                               'tensorflow/logs/soft_actor_critic')
expm_name = 'CartPole-v0'

base_params = params[expm_name]['base']
env = gym.make(base_params['env_name'])

n_inputs = env.observation_space.shape[0]
n_outputs = env.action_space.n
    
epoch_length = base_params['epoch_length']
max_epochs = base_params['max_epochs']
online_training = base_params['online_training']
grad_steps_per_t = base_params['grad_steps_per_t']

lr = params[expm_name]['algorithm']['lr']
reward_scale = params[expm_name]['algorithm']['reward_scale']
tau = params[expm_name]['algorithm']['tau']
epsilon_start = params[expm_name]['algorithm']['epsilon_start']
epsilon_end = params[expm_name]['algorithm']['epsilon_end']
epsilon_decay = params[expm_name]['algorithm']['epsilon_decay']

scheme = params[expm_name]['algorithm']['scheme']

n_hidden = params[expm_name]['nnet']['n_hidden']
n_layers = params[expm_name]['nnet']['n_layers']

batch_size = params[expm_name]['replay_buffer']['batch_size']
max_buffer_size = params[expm_name]['replay_buffer']['max_buffer_size']
min_pool_size = params[expm_name]['replay_buffer']['min_pool_size']
# Todo make these into lists so that can define each layer separately


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [111]:
tf.reset_default_graph() # THIS IS NECESSARY BEFORE MAKING NEW SESSION TO STOP IT ERRORING!!
try:
    sess
except:
    pass
else:
    sess.close()
    del sess
sess = tf.InteractiveSession()

rewards = tf.placeholder(tf.float32,shape = [None],name = 'rewards')
actions = tf.placeholder(tf.float32,shape = [None,n_outputs],name = 'actions')
observations = tf.placeholder(tf.float32,shape = [None,n_inputs],name = 'observations')
next_observations = tf.placeholder(tf.float32,shape = [None,n_inputs],name = 'next_observations')
dones = tf.placeholder(tf.float32,shape = [None],name = 'dones')

qnet = Qnet(n_outputs,observations,n_hidden=n_hidden,n_layers=n_layers)
policy = Policy_Discrete(qnet,reward_scale=reward_scale,epsilon_start = epsilon_start,epsilon_end=epsilon_end,epsilon_decay=epsilon_decay,scheme=scheme)

algo = Deep_Q_Learning(qnet,actions,observations,next_observations,rewards,dones,lr=lr,tau=tau)

rb = replayBuffer(n_inputs,n_outputs,max_buffer_size,min_pool_size = min_pool_size,batch_size=batch_size)
sampler = Sampler(policy,env,rb)

            
log = logger() 
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter(log_dir, sess.graph)

tf.global_variables_initializer().run()


In [116]:
for i in range(max_epochs):
    sampler.reset()
    epoch_avg_losses = 0
    
    for t in range(epoch_length):
        sampler.sample()
        
        if rb.batch_ready() or online_training:
            if online_training:
                samples = rb.get_last_sample()
                summary,losses,qnet_o = algo.train(samples,merged,algo.Q_Loss,qnet.output) 
            else:
                for j in range(grad_steps_per_t):
                    samples = rb.get_samples()
                    summary,losses= algo.train(samples,merged,algo.Q_Loss) 
            epoch_avg_losses = (epoch_avg_losses*(t) + np.array(losses))/(t+1)
            print(algo.predict.eval(feed_dict = algo._construct_feed_dict(samples)))
            print(algo.next_obs_out.eval(feed_dict = algo._construct_feed_dict(samples)))
            print(algo.target_Q_outputs.eval(feed_dict = algo._construct_feed_dict(samples)))

    log.record('mean_episode_reward',sampler.mean_episode_reward)
    writer.add_summary(summary, i)
    print(epoch_avg_losses)
    
    writer.flush()

    print('Epoch %i, mean_reward %d' % (i, sampler.mean_episode_reward))
    


[[0. 1.]]
[[0.15013374 0.31614375]]
[[0.14823645 0.31426245]]
[[0. 1.]]
[[0.05562652 0.05651041]]
[[0.05434782 0.05519848]]
[[0. 1.]]
[[0.16329876 0.33569828]]
[[0.1613797 0.3337908]]
[[0. 1.]]
[[0.06893576 0.07667284]]
[[0.06734742 0.07504582]]
[[0. 1.]]
[[0.17836563 0.35799688]]
[[0.1764203  0.35605735]]
[[0. 1.]]
[[0.07950319 0.09492174]]
[[0.07788451 0.09326538]]
[[0. 1.]]
[[0.19568157 0.3835454 ]]
[[0.19370484 0.38156736]]
[[0. 1.]]
[[0.09164723 0.1158376 ]]
[[0.08999173 0.11414463]]
[[0. 1.]]
[[0.21564442 0.41292265]]
[[0.21363029 0.41089877]]
[[0. 1.]]
[[0.10565613 0.1399047 ]]
[[0.1039568  0.13816707]]
[[0. 1.]]
[[0.23871145 0.44679272]]
[[0.2366529  0.44471434]]
[[0. 1.]]
[[0.34614626 0.7286211 ]]
[[0.34358937 0.7260412 ]]
[[0. 1.]]
[[0.26690066 0.4869524 ]]
[[0.26477724 0.48480716]]
[[0. 1.]]
[[0.13867298 0.19836977]]
[[0.13684756 0.19650853]]
[[0. 1.]]
[[0.29745975 0.5335795 ]]
[[0.29507953 0.53115785]]
[[0. 1.]]
[[0.16337283 0.24030346]]
[[0.1614692  0.23835662]]
[[1. 0.]]


[[1. 0.]]
[[ 0.11289304 -0.09253321]]
[[ 0.11159521 -0.09386926]]
[[0. 1.]]
[[0.01419898 0.17763071]]
[[0.01282995 0.17634988]]
[[1. 0.]]
[[ 0.1259745  -0.15051994]]
[[ 0.12461776 -0.15167683]]
[[1. 0.]]
[[ 0.21977086 -0.40270224]]
[[ 0.21817116 -0.40408164]]
[[1. 0.]]
[[ 0.14668702 -0.21404806]]
[[ 0.14524563 -0.21527566]]
[[1. 0.]]
[[ 0.11218031 -0.22380899]]
[[ 0.11052261 -0.2252783 ]]
[[1. 0.]]
[[0.04460305 0.01692989]]
[[0.0437563 0.0160435]]
[[0. 1.]]
[[0.12698068 0.28645188]]
[[0.12509051 0.28481013]]
[[0. 1.]]
[[0.220216  0.5465384]]
[[0.21776974 0.5445319 ]]
[[0. 1.]]
[[0.31483856 0.808133  ]]
[[0.3118332  0.80576587]]
[[0. 1.]]
[[0.41173512 1.0725689 ]]
[[0.40815845 1.0698432 ]]
[[0. 1.]]
[[0.31999364 0.81208265]]
[[0.31697246 0.8097222 ]]
[[0. 1.]]
[[0.4226828 1.0853065]]
[[0.41906878 1.0825651 ]]
[[0. 1.]]
[[0.33698955 0.8339802 ]]
[[0.3339102  0.83158535]]
[[0. 1.]]
[[0.44531432 1.1156651 ]]
[[0.4416187 1.1128621]]
[[0. 1.]]
[[0.5566675 1.4011917]]
[[0.5523253 1.3979712]]


[[0. 1.]]
[[0.47612727 0.8817859 ]]
[[0.471855  0.8788817]]
[[0. 1.]]
[[0.15821119 0.2893626 ]]
[[0.15583971 0.28758296]]
[[0. 1.]]
[[0.24963826 0.5448988 ]]
[[0.24645638 0.5426303 ]]
[[0. 1.]]
[[0.34256655 0.8021258 ]]
[[0.33856612 0.7993632 ]]
[[0. 1.]]
[[0.4378856 1.0623773]]
[[0.43304962 1.059114  ]]
[[0. 1.]]
[[0.5364624 1.3269348]]
[[0.5307625 1.3231621]]
[[0. 1.]]
[[0.4439736 1.0661781]]
[[0.43906683 1.0628802 ]]
[[0. 1.]]
[[0.5501702 1.3422196]]
[[0.54435945 1.3383735 ]]
[[0. 1.]]
[[0.66028553 1.6234329 ]]
[[0.653525  1.6190237]]
[[0. 1.]]
[[0.77508026 1.9108727 ]]
[[0.76730424 1.9058831 ]]
[[0. 1.]]
[[0.7024096 1.6784625]]
[[0.6953135 1.6738741]]
[[0. 1.]]
[[0.6363002 1.4565766]]
[[0.6298858 1.4523652]]
[[0. 1.]]
[[0.57609624 1.2441504 ]]
[[0.5703637 1.2403461]]
[[0. 1.]]
[[0.18735446 0.33147636]]
[[0.18474075 0.32960117]]
[[0. 1.]]
[[0.28111452 0.5902461 ]]
[[0.27761784 0.58782166]]
[[0. 1.]]
[[0.37656164 0.8509651 ]]
[[0.37216458 0.8479795 ]]
[[0. 1.]]
[[0.47458127 1.1149571

[[0.19268772 0.34839824]]
[[1. 0.]]
[[0.08057987 0.07473578]]
[[0.0785917  0.07343215]]
[[1. 0.]]
[[ 0.04811684 -0.21895623]]
[[ 0.04672626 -0.21964736]]
[[1. 0.]]
[[0.09022015 0.08992152]]
[[0.08819415 0.08857944]]
[[1. 0.]]
[[ 0.04812638 -0.21424532]]
[[ 0.04673149 -0.21493654]]
[[1. 0.]]
[[ 0.08122033 -0.49268863]]
[[ 0.07964781 -0.4935279 ]]
[[1. 0.]]
[[ 0.05851935 -0.21392451]]
[[ 0.0571168  -0.21462727]]
[[0. 1.]]
[[0.1080366  0.11747836]]
[[0.10596867 0.11607249]]
[[1. 0.]]
[[ 0.06011019 -0.21191706]]
[[ 0.0586923  -0.21262906]]
[[0. 1.]]
[[0.1118366  0.12331992]]
[[0.10976335 0.1218898 ]]
[[1. 0.]]
[[ 0.06178309 -0.21015008]]
[[ 0.06034759 -0.21087107]]
[[0. 1.]]
[[0.11508305 0.12827903]]
[[0.11300771 0.12682477]]
[[0. 1.]]
[[0.23157865 0.4044547 ]]
[[0.22842436 0.40249985]]
[[0. 1.]]
[[0.33338457 0.6734294 ]]
[[0.32921064 0.6708914 ]]
[[0. 1.]]
[[0.43710575 0.944643  ]]
[[0.43191078 0.9415235 ]]
[[0. 1.]]
[[0.5436267 1.2193977]]
[[0.53739804 1.2156957 ]]
[[0. 1.]]
[[0.65379566

[[ 0.10486884 -0.25008625]]
[[ 0.10339646 -0.25078332]]
[[1. 0.]]
[[0.10146578 0.05381289]]
[[0.09971051 0.05264825]]
[[0. 1.]]
[[0.18875167 0.33744273]]
[[0.18610384 0.33547792]]
[[0. 1.]]
[[0.2854422  0.59790087]]
[[0.28195345 0.5953032 ]]
[[0. 1.]]
[[0.38380888 0.8602768 ]]
[[0.37947494 0.8570469 ]]
[[0. 1.]]
[[0.48475218 1.1259046 ]]
[[0.47955966 1.1220406 ]]
[[0. 1.]]
[[0.58914614 1.3960596 ]]
[[0.58306926 1.3915561 ]]
[[0. 1.]]
[[0.69782305 1.6719297 ]]
[[0.6908206 1.666778 ]]
[[0. 1.]]
[[0.8115538 1.9545802]]
[[0.8035655 1.9487679]]
[[0. 1.]]
[[0.93102473 2.2449117 ]]
[[0.9219673 2.2384224]]
[[0. 1.]]
[[1.0568073 2.5436056]]
[[1.0465707 2.5364192]]
[[0. 1.]]
[[1.1892139 2.8510497]]
[[1.1777645 2.8431551]]
[[0. 1.]]
[[0.17948097 0.35454842]]
[[0.17661154 0.35257316]]
[[0. 1.]]
[[0.27853462 0.61834985]]
[[0.27470753 0.6157619 ]]
[[0. 1.]]
[[0.37930256 0.884071  ]]
[[0.37448928 0.8808701 ]]
[[0. 1.]]
[[0.4826821 1.153043 ]]
[[0.4768483 1.1492269]]
[[0. 1.]]
[[0.5895423 1.426534 ]]


...

[[0. 1.]]
[[0.5421059 1.1350802]]
[[0.53900945 1.1317797 ]]
[[0. 1.]]
[[0.44603765 0.8821097 ]]
[[0.44334617 0.8792113 ]]
[[0. 1.]]
[[0.58051336 1.1866062 ]]
[[0.5773041 1.1831769]]
[[0. 1.]]
[[0.71813434 1.4948378 ]]
[[0.71437824 1.4908636 ]]
[[0. 1.]]
[[0.63426447 1.2585719 ]]
[[0.6308762 1.2549683]]
[[0. 1.]]
[[0.55547696 1.03025   ]]
[[0.55245805 1.0270091 ]]
[[0. 1.]]
[[0.70306325 1.3530324 ]]
[[0.6994773 1.3491834]]
[[1. 0.]]
[[ 0.13058615 -0.17570278]]
[[ 0.12887046 -0.17737827]]
[[1. 0.]]
[[ 0.17301868 -0.45025218]]
[[ 0.17087238 -0.45239973]]
[[1. 0.]]
[[ 0.1437219 -0.1791136]]
[[ 0.14201412 -0.18079227]]
[[0. 1.]]
[[0.11301406 0.11712465]]
[[0.1117214  0.11559539]]
[[1. 0.]]
[[ 0.14851758 -0.1812954 ]]
[[ 0.14680669 -0.1829707 ]]
[[1. 0.]]
[[0.11253817 0.11085607]]
[[0.11129387 0.10927917]]
[[0. 1.]]
[[0.23583113 0.39220542]]
[[0.23395301 0.390104  ]]
[[0. 1.]]
[[0.3496741  0.66788214]]
[[0.34731564 0.66525424]]
[[0. 1.]]
[[0.46536985 0.94561666]]
[[0.46252123 0.94245696]]
[[

[[0. 1.]]
[[0.68641895 1.3267465 ]]
[[0.68109834 1.3220568 ]]
[[0. 1.]]
[[0.8363407 1.6512961]]
[[0.8300918 1.645741 ]]
[[1. 0.]]
[[ 0.1394946  -0.16409567]]
[[ 0.1381731  -0.16490218]]
[[0. 1.]]
[[0.07927113 0.08393457]]
[[0.07768061 0.08256219]]
[[1. 0.]]
[[ 0.14488542 -0.16857776]]
[[ 0.14375249 -0.16918783]]
[[1. 0.]]
[[ 0.19027624 -0.44522068]]
[[ 0.18877174 -0.4461622 ]]
[[1. 0.]]
[[ 0.15490715 -0.18560027]]
[[ 0.15376803 -0.1862222 ]]
[[1. 0.]]
[[0.07551041 0.05783893]]
[[0.07392054 0.05655281]]
[[0. 1.]]
[[0.18130533 0.34035093]]
[[0.17884597 0.33823287]]
[[1. 0.]]
[[0.06750788 0.03671671]]
[[0.06595422 0.03546897]]
[[1. 0.]]
[[ 0.16102423 -0.2041838 ]]
[[ 0.1598573  -0.20481122]]
[[1. 0.]]
[[ 0.22073463 -0.47078997]]
[[ 0.21918346 -0.47174996]]
[[1. 0.]]
[[ 0.17383379 -0.22817735]]
[[ 0.1726522  -0.22882318]]
[[1. 0.]]
[[ 0.09843157 -0.00619698]]
[[ 0.09757992 -0.00689325]]
[[0. 1.]]
[[0.13408452 0.2756481 ]]
[[0.13185808 0.27369207]]
[[0. 1.]]
[[0.23941013 0.5388075 ]]
[[0.23

[[0.8204534 1.5620668]]
[[0.81393725 1.5570978 ]]
[[0. 1.]]
[[0.24403119 0.38957134]]
[[0.24131441 0.3874913 ]]
[[0. 1.]]
[[0.36120647 0.668334  ]]
[[0.35757846 0.6655593 ]]
[[0. 1.]]
[[0.48023015 0.9491211 ]]
[[0.47567818 0.9456421 ]]
[[0. 1.]]
[[0.36742306 0.67387956]]
[[0.36375144 0.6710679 ]]
[[0. 1.]]
[[0.49150363 0.9619502 ]]
[[0.48688856 0.9584098 ]]
[[0. 1.]]
[[0.6183578 1.2533239]]
[[0.61277837 1.2490426 ]]
[[0. 1.]]
[[0.7489116 1.5492924]]
[[0.7423346 1.5442561]]
[[0. 1.]]
[[0.8840325 1.851036 ]]
[[0.8764082 1.845227 ]]
[[0. 1.]]
[[1.0245026 2.1595778]]
[[1.0157611 2.1529768]]
[[0. 1.]]
[[1.1709888 2.4757342]]
[[1.1610359 2.4683166]]
[[0. 1.]]
[[1.0958675 2.2505393]]
[[1.0865684 2.2437975]]
[[0. 1.]]
[[0.21678881 0.37970567]]
[[0.21393842 0.3776755 ]]
[[1. 0.]]
[[0.09248925 0.0807819 ]]
[[0.09079534 0.07953814]]
[[0. 1.]]
[[0.20593864 0.36338863]]
[[0.20310639 0.36142385]]
[[0. 1.]]
[[0.31872603 0.6357302 ]]
[[0.31490946 0.63314337]]
[[0. 1.]]
[[0.4332176  0.90988296]]
[[0.42

[[0.8023345 1.7175357]]
[[0. 1.]]
[[0.71929395 1.4790967 ]]
[[0.7130158 1.4755113]]
[[0. 1.]]
[[0.8683328 1.798863 ]]
[[0.8609777 1.7947142]]
[[0. 1.]]
[[0.79040825 1.5735954 ]]
[[0.78378797 1.5698185 ]]
[[0. 1.]]
[[0.22429289 0.3376721 ]]
[[0.22199613 0.33615285]]
[[0. 1.]]
[[0.34277585 0.6159597 ]]
[[0.33935878 0.61381614]]
[[0. 1.]]
[[0.22229703 0.33329618]]
[[0.21999885 0.33178213]]
[[1. 0.]]
[[0.11328621 0.03077502]]
[[0.11181368 0.02964003]]
[[0. 1.]]
[[0.22481279 0.33644557]]
[[0.22250606 0.33492422]]
[[0. 1.]]
[[0.34781682 0.6206976 ]]
[[0.34438404 0.61853933]]
[[0. 1.]]
[[0.2298477  0.34222457]]
[[0.22751981 0.34069026]]
[[1. 0.]]
[[0.10331368 0.052678  ]]
[[0.1015718  0.05130849]]
[[1. 0.]]
[[ 0.1145981  -0.23306884]]
[[ 0.11278467 -0.23456669]]
[[1. 0.]]
[[0.11288171 0.06610873]]
[[0.11112397 0.06461135]]
[[0. 1.]]
[[0.24799308 0.36822146]]
[[0.2456204  0.36664128]]
[[0. 1.]]
[[0.37467986 0.65709513]]
[[0.37119767 0.65486836]]
[[0. 1.]]
[[0.50056237 0.9455712 ]]
[[0.49616033

KeyboardInterrupt: 

[[0. 1.]]
[[0.02752121 0.25922558]]
[[0.02610591 0.25770772]]
[[0.00681379 0.05556605]]


array([[2.0556815, 0.5648127]], dtype=float32)